# Herramientas para Fundamentación sobre la Reforma al Poder Judicial y Organismos Autónomos

In [1]:
from googlesearch import search
import requests
import os
from bs4 import BeautifulSoup
import pdfplumber
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Herramienta para extraer PDFS a partir de palabras clave Reforma al Poder Judicial

In [2]:
def search_pdfs(keywords, num_results, allowed_domains):
    query = f"{keywords} filetype:pdf"
    urls = []
    other_urls = []

    for url in search(query, num_results=num_results, lang="es"):
        if url.endswith(".pdf"):
            if allowed_domains and any(domain in url for domain in allowed_domains):
                urls.append(url) 
            else:
                other_urls.append(url) 

    return urls + other_urls

def download_pdf(url, output_dir):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        filename = os.path.join(output_dir, url.split("/")[-1])
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Descargado: {filename}")
        return True
    except requests.RequestException as e:
        print(f"Error al descargar {url}: {e}")
        return False

if __name__ == "__main__":
    output_dir = "./documentos"
    os.makedirs(output_dir, exist_ok=True)
    
    KEYWORDS = [        
        "Diagnóstico reforma poder judicial México 2024 estudios expertos",
        "Reforma judicial México exclusión fiscalías defensoría",
        "Medidas crimen organizado violencia electoral reforma judicial",
        "Protección juristas seguridad violencia reforma judicial",
        "Comités de postulación reforma judicial proceso selección",
        "Carrera judicial México reforma mecanismos aseguramiento",
        "Jueces sin rostro estándares interamericanos protección identidad",
        "Costo económico reforma judicial impacto acceso justicia",
        "Reforma al Poder Judicial México", 
    ]

    total_files_to_download = 30  
    allowed_domains = ["scjn.gob.mx", "dof.gob.mx", "senado.gob.mx", "diputados.gob.mx", "latinus.us"]

    print("Buscando PDFs...")
    downloaded_files = 0
    attempts = 0
    max_attempts = 60  

    while downloaded_files < total_files_to_download and attempts < max_attempts:
        
        attempts += 1
        print(f"\nIntento {attempts}: Buscando PDFs...")

        
        for keyword in KEYWORDS:
            print(f"\nBuscando para la palabra clave: {keyword}")

            
            if keyword == "Reforma al Poder Judicial México":
                num_results = 30  
            else:
                num_results = 1

            pdf_urls = search_pdfs(keyword, num_results=num_results, allowed_domains=allowed_domains)

            if not pdf_urls:
                print("No se encontraron resultados para esta palabra clave.")
                continue

            for pdf_url in pdf_urls:
                if downloaded_files >= total_files_to_download:
                    break
                success = download_pdf(pdf_url, output_dir)
                if success:
                    downloaded_files += 1

        if downloaded_files >= total_files_to_download:
            break

    if downloaded_files < total_files_to_download:
        print(f"Descarga incompleta. Se descargaron {downloaded_files} de {total_files_to_download} archivos.")
    else:
        print(f"Descarga completa. Se descargaron {total_files_to_download} archivos en '{output_dir}'.")

Buscando PDFs...

Intento 1: Buscando PDFs...

Buscando para la palabra clave: Diagnóstico reforma poder judicial México 2024 estudios expertos
Descargado: ./documentos\Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf

Buscando para la palabra clave: Reforma judicial México exclusión fiscalías defensoría
Descargado: ./documentos\Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf

Buscando para la palabra clave: Medidas crimen organizado violencia electoral reforma judicial
Descargado: ./documentos\Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf

Buscando para la palabra clave: Protección juristas seguridad violencia reforma judicial
Descargado: ./documentos\Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf

Buscando para la palabra clave: Comités de postulación reforma judicial proceso selección
Descargado: ./documentos\convocatoria-c

Herramienta para scrapear paginas web con palabras clave y volverlas en txt

In [3]:
def buscar_articulos(keywords, num_resultados, allowed_domains):
    
    query = keywords
    urls = []
    other_urls = []

    try:
        for url in search(query, num_results=num_resultados * 2, lang="es"):
            if allowed_domains and any(domain in url for domain in allowed_domains):
                urls.append(url)  
            else:
                other_urls.append(url)  
                
        return urls[:num_resultados] + other_urls[:num_resultados]
    except Exception as e:
        print(f"Error al buscar artículos: {e}")
        return []

def extraer_texto_principal(url):
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        contenedor = soup.find(['article', 'div'], class_=["main-content", "content", "post-content"])
        if not contenedor:
            contenedor = soup

        for tag in contenedor.find_all(['aside', 'nav', 'footer', 'script', 'style']):
            tag.decompose()
        for no_relevante in contenedor.find_all(['div', 'span'], class_=['ads', 'promo', 'banner']):
            no_relevante.decompose()

        texto = "\n".join(tag.get_text(strip=True) for tag in contenedor.find_all(['h1', 'h2', 'h3', 'p']))
        return texto if texto.strip() else None

    except requests.RequestException as e:
        print(f"Error en URL: {url}: {e}")
        return None

def guardar_texto(texto, ruta):
    
    try:
        with open(ruta, "w", encoding="utf-8") as archivo:
            archivo.write(texto)
        print(f"Guardado en: {ruta}")
    except Exception as e:
        print(f"Error al guardar: {e}")

if __name__ == "__main__":
    
    directorio = "./documentos/"
    os.makedirs(directorio, exist_ok=True)

    KEYWORDS = [
        "Diagnóstico reforma poder judicial México 2024 estudios expertos",
        "Reforma judicial México exclusión fiscalías defensoría",
        "Medidas crimen organizado violencia electoral reforma judicial",
        "Protección juristas seguridad violencia reforma judicial",
        "Comités de postulación reforma judicial proceso selección",
        "Carrera judicial México reforma mecanismos aseguramiento",
        "Jueces sin rostro estándares interamericanos protección identidad",
        "Costo económico reforma judicial impacto acceso justicia"
        "Reforma al Poder Judicial México",  
    ]

    allowed_domains = ["scjn.gob.mx", "dof.gob.mx", "senado.gob.mx", "diputados.gob.mx", "latinus.us", "jornada.com.mx", "eluniversal.com.mx"]
    total_articulos_a_descargar = 30
    max_intentos = 60

    print("Buscando artículos...")
    descargados = 0
    intentos = 0

    while descargados < total_articulos_a_descargar and intentos < max_intentos:
        intentos += 1
        print(f"\nIntento {intentos}: Buscando artículos...")
        
        for i, keyword in enumerate(KEYWORDS):
            print(f"\nBuscando para la palabra clave: {keyword}")
            
            if keyword == "Reforma al Poder Judicial México":
                
                num_resultados = total_articulos_a_descargar - descargados
            else:
                num_resultados = 1

            urls = buscar_articulos(keyword, num_resultados, allowed_domains)

            if not urls:
                print("No se encontraron artículos para esta palabra clave.")
                continue

            for url in urls:
                if descargados >= total_articulos_a_descargar:
                    break

                print(f"Procesando artículo {descargados + 1}: {url}")
                texto = extraer_texto_principal(url)
                if texto:
                    ruta_archivo = os.path.join(directorio, f"articulo_Juducial_{descargados + 1}.txt")
                    guardar_texto(texto, ruta_archivo)
                    descargados += 1

        if descargados >= total_articulos_a_descargar:
            break

    if descargados < total_articulos_a_descargar:
        print(f"Descarga incompleta. Se descargaron {descargados} de {total_articulos_a_descargar} artículos.")
    else:
        print(f"Descarga completa. Se descargaron {total_articulos_a_descargar} artículos en '{directorio}'.")


Buscando artículos...

Intento 1: Buscando artículos...

Buscando para la palabra clave: Diagnóstico reforma poder judicial México 2024 estudios expertos
Procesando artículo 1: https://www.sitios.scjn.gob.mx/cec/sites/default/files/page/files/2024-06/Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Procesando artículo 1: https://www.eld.edu.mx/Biblioteca/Estudio-sobre-la-Reforma-Judicial-version-completa.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Guardado en: ./documentos/articulo_Juducial_1.txt

Buscando para la palabra clave: Reforma judicial México exclusión fiscalías defensoría
Procesando artículo 2: https://www.sitios.scjn.gob.mx/cec/sites/default/files/page/files/2024-06/Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf


...

Procesando artículo 29: https://www.eld.edu.mx/Biblioteca/Estudio-sobre-la-Reforma-Judicial-version-completa.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Guardado en: ./documentos/articulo_Juducial_29.txt

Buscando para la palabra clave: Reforma judicial México exclusión fiscalías defensoría
Procesando artículo 30: https://www.sitios.scjn.gob.mx/cec/sites/default/files/page/files/2024-06/Ana%CC%81lisis%20de%20la%20iniciativa%20de%20reforma.%20Problemas%20asociados_final.pdf


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



Buscando para la palabra clave: Medidas crimen organizado violencia electoral reforma judicial
Procesando artículo 30: https://animalpolitico.com/politica/eleccion-jueces-reforma-judicial-crimen-violencia
Guardado en: ./documentos/articulo_Juducial_30.txt

Buscando para la palabra clave: Protección juristas seguridad violencia reforma judicial

Buscando para la palabra clave: Comités de postulación reforma judicial proceso selección

Buscando para la palabra clave: Carrera judicial México reforma mecanismos aseguramiento

Buscando para la palabra clave: Jueces sin rostro estándares interamericanos protección identidad

Buscando para la palabra clave: Costo económico reforma judicial impacto acceso justiciaReforma al Poder Judicial México
Descarga completa. Se descargaron 30 artículos en './documentos/'.


## Herramienta para extraer PDFS a partir de palabras clave Reforma a los organismos autónomos

In [4]:
def search_pdfs(keywords, num_results, allowed_domains):
    
    query = f"{keywords} filetype:pdf"
    urls = []
    other_urls = []

    for url in search(query, num_results=num_results, lang="es"):
        if url.endswith(".pdf"):
            if allowed_domains and any(domain in url for domain in allowed_domains):
                urls.append(url)  
            else:
                other_urls.append(url)  

    return urls + other_urls  

def download_pdf(url, output_dir):
    
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        filename = os.path.join(output_dir, url.split("/")[-1])
        with open(filename, "wb") as file:
            for chunk in response.iter_content(chunk_size=1024):
                if chunk:
                    file.write(chunk)
        print(f"Descargado: {filename}")
        return True
    except requests.RequestException as e:
        print(f"Error al descargar {url}: {e}")
        return False

if __name__ == "__main__":
    
    output_dir = "./documentos"
    os.makedirs(output_dir, exist_ok=True)

    KEYWORDS = [        
        "Constitucionalidad de la ley eliminación organismos autónomos México",
        "Ley eliminación organismos autónomos y Constitución México",
        "Organismos autónomos establecidos en la Constitución México",
        "Impacto constitucional de la reforma a organismos autónomos",
        "Eliminación organismos autónomos transparencia rendición de cuentas México",
        "Consecuencias de eliminar organismos autónomos para la transparencia",
        "Reforma organismos autónomos y transparencia gubernamental",
        "Pérdida de independencia funciones organismos autónomos",
        "Reformas y control de organismos autónomos por el poder ejecutivo",
        "Consecuencias de centralizar funciones críticas en el poder ejecutivo",
        "Alternativas eficiencia organismos autónomos sin eliminación",
        "Mejorar eficiencia organismos autónomos México",
        "Propuestas de mejora eficiencia organismos autónomos en México",
        "Sectores afectados por la desaparición de organismos autónomos México",
        "Impacto de eliminar organismos autónomos en la sociedad civil"
        "Impacto de la desaparición de organismos autónomos en derechos civiles",
        "Reforma a los organismos autónomos México", 
    ]


    total_files_to_download = 30  
    allowed_domains = ["scjn.gob.mx", "dof.gob.mx", "senado.gob.mx", "diputados.gob.mx", "latinus.us"]  

    print("Buscando PDFs...")
    downloaded_files = 30
    attempts = 0
    max_attempts = 60  

    while downloaded_files < total_files_to_download and attempts < max_attempts:
        
        attempts += 1
        print(f"\nIntento {attempts}: Buscando PDFs...")

        for keyword in KEYWORDS:
            print(f"\nBuscando para la palabra clave: {keyword}")

            if keyword == "Reforma a los organismos autónomos México":
                num_results = 30 
            else:
                num_results = 1

            pdf_urls = search_pdfs(keyword, num_results=num_results, allowed_domains=allowed_domains)

            if not pdf_urls:
                print("No se encontraron resultados para esta palabra clave.")
                continue

            for pdf_url in pdf_urls:
                if downloaded_files >= total_files_to_download:
                    break
                success = download_pdf(pdf_url, output_dir)
                if success:
                    downloaded_files += 1

        if downloaded_files >= total_files_to_download:
            break

    if downloaded_files < total_files_to_download:
        print(f"Descarga incompleta. Se descargaron {downloaded_files} de {total_files_to_download} archivos.")
    else:
        print(f"Descarga completa. Se descargaron {total_files_to_download} archivos en '{output_dir}'.")

Buscando PDFs...
Descarga completa. Se descargaron 30 archivos en './documentos'.


Herramienta para scrapear paginas web con palabras clave y volverlas en txt

In [5]:
def buscar_articulos(keywords, num_resultados, allowed_domains=None):
    
    query = keywords
    urls = []
    other_urls = []

    try:
        for url in search(query, num_results=num_resultados * 2, lang="es"):
            if allowed_domains and any(domain in url for domain in allowed_domains):
                urls.append(url)  
            else:
                other_urls.append(url)  

        return urls[:num_resultados] + other_urls[:num_resultados]
    except Exception as e:
        print(f"Error al buscar artículos: {e}")
        return []

def extraer_texto_principal(url):
    
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, "html.parser")
        
        contenedor = soup.find(['article', 'div'], class_=["main-content", "content", "post-content"])
        if not contenedor:
            contenedor = soup

        for tag in contenedor.find_all(['aside', 'nav', 'footer', 'script', 'style']):
            tag.decompose()
        for no_relevante in contenedor.find_all(['div', 'span'], class_=['ads', 'promo', 'banner']):
            no_relevante.decompose()

        texto = "\n".join(tag.get_text(strip=True) for tag in contenedor.find_all(['h1', 'h2', 'h3', 'p']))
        return texto if texto.strip() else None

    except requests.RequestException as e:
        print(f"Error en URL: {url}: {e}")
        return None

def guardar_texto(texto, ruta):
    
    try:
        with open(ruta, "w", encoding="utf-8") as archivo:
            archivo.write(texto)
        print(f"Guardado en: {ruta}")
    except Exception as e:
        print(f"Error al guardar: {e}")

if __name__ == "__main__":
    
    directorio = "./documentos/"
    os.makedirs(directorio, exist_ok=True)

    KEYWORDS = [
        "Constitucionalidad de la ley eliminación organismos autónomos México",
        "Ley eliminación organismos autónomos y Constitución México",
        "Organismos autónomos establecidos en la Constitución México",
        "Impacto constitucional de la reforma a organismos autónomos",
        "Eliminación organismos autónomos transparencia rendición de cuentas México",
        "Consecuencias de eliminar organismos autónomos para la transparencia",
        "Reforma organismos autónomos y transparencia gubernamental",
        "Pérdida de independencia funciones organismos autónomos",
        "Reformas y control de organismos autónomos por el poder ejecutivo",
        "Consecuencias de centralizar funciones críticas en el poder ejecutivo",
        "Alternativas eficiencia organismos autónomos sin eliminación",
        "Mejorar eficiencia organismos autónomos México",
        "Propuestas de mejora eficiencia organismos autónomos en México",
        "Sectores afectados por la desaparición de organismos autónomos México",
        "Impacto de eliminar organismos autónomos en la sociedad civil"
        "Impacto de la desaparición de organismos autónomos en derechos civiles",
        "Reforma a los organismos autónomos México", 
    ]

    allowed_domains = ["scjn.gob.mx", "dof.gob.mx", "senado.gob.mx", "diputados.gob.mx", "latinus.us", "jornada.com.mx", "eluniversal.com.mx"]
    total_articulos_a_descargar = 30
    max_intentos = 60

    print("Buscando artículos...")
    descargados = 0
    intentos = 0

    while descargados < total_articulos_a_descargar and intentos < max_intentos:
        intentos += 1
        print(f"\nIntento {intentos}: Buscando artículos...")
        
        for i, keyword in enumerate(KEYWORDS):
            print(f"\nBuscando para la palabra clave: {keyword}")
            
            if keyword == "Reforma a los organismos autónomos México":
                
                num_resultados = total_articulos_a_descargar - descargados
            else:
                
                num_resultados = 1

            urls = buscar_articulos(keyword, num_resultados, allowed_domains)

            if not urls:
                print("No se encontraron artículos para esta palabra clave.")
                continue

            for url in urls:
                if descargados >= total_articulos_a_descargar:
                    break

                print(f"Procesando artículo {descargados + 1}: {url}")
                texto = extraer_texto_principal(url)
                if texto:
                    ruta_archivo = os.path.join(directorio, f"articulo_Autonomo_{descargados + 1}.txt")
                    guardar_texto(texto, ruta_archivo)
                    descargados += 1

        if descargados >= total_articulos_a_descargar:
            break

    if descargados < total_articulos_a_descargar:
        print(f"Descarga incompleta. Se descargaron {descargados} de {total_articulos_a_descargar} artículos.")
    else:
        print(f"Descarga completa. Se descargaron {total_articulos_a_descargar} artículos en '{directorio}'.")


Buscando artículos...

Intento 1: Buscando artículos...

Buscando para la palabra clave: Constitucionalidad de la ley eliminación organismos autónomos México
Procesando artículo 1: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-
Error en URL: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-: HTTPSConnectionPool(host='comunicacionsocial.diputados.gob.mx', port=443): Max retries exceeded with url: /index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos- (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Procesando artículo 1: https://archivos.juridicas.unam.mx/w

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



Buscando para la palabra clave: Ley eliminación organismos autónomos y Constitución México
Procesando artículo 1: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-
Error en URL: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-: HTTPSConnectionPool(host='comunicacionsocial.diputados.gob.mx', port=443): Max retries exceeded with url: /index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos- (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Procesando artículo 1: https://cnnespanol.cnn.com/2024/11/29/senado-mexico-aprueba-reforma-eliminacion-siete-organos-autonomo

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Procesando artículo 15: https://cnnespanol.cnn.com/2024/11/29/senado-mexico-aprueba-reforma-eliminacion-siete-organos-autonomos-orix
Guardado en: ./documentos/articulo_Autonomo_15.txt
Procesando artículo 16: https://eljuegodelacorte.nexos.com.mx/el-abc-de-la-reforma-a-los-organismos-autonomos/
Error en URL: https://eljuegodelacorte.nexos.com.mx/el-abc-de-la-reforma-a-los-organismos-autonomos/: 403 Client Error: Forbidden for url: https://eljuegodelacorte.nexos.com.mx/el-abc-de-la-reforma-a-los-organismos-autonomos/
Procesando artículo 16: https://coparmex.org.mx/que-perdemos-con-la-desaparicion-de-los-organismos-autonomos-derechos-transparencia-y-prograso/
Guardado en: ./documentos/articulo_Autonomo_16.txt
Procesando artículo 17: https://elpais.com/mexico/2024-08-28/organos-autonomos-que-son-y-que-implica-la-reforma-para-desaparecerlos.html
Guardado en: ./documentos/articulo_Autonomo_17.txt
Procesando artículo 18: https://apnews.com/article/mexico-senado-reforma-constitucion-autonomos-

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


Procesando artículo 26: https://verificado.com.mx/organismos-autonomos-desaparecen/
Error en URL: https://verificado.com.mx/organismos-autonomos-desaparecen/: HTTPSConnectionPool(host='verificado.com.mx', port=443): Read timed out. (read timeout=10)

Intento 2: Buscando artículos...

Buscando para la palabra clave: Constitucionalidad de la ley eliminación organismos autónomos México
Procesando artículo 26: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-
Error en URL: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-: HTTPSConnectionPool(host='comunicacionsocial.diputados.gob.mx', port=443): Max retries exceeded with url: /index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonom

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.



Buscando para la palabra clave: Ley eliminación organismos autónomos y Constitución México
Procesando artículo 26: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-
Error en URL: https://comunicacionsocial.diputados.gob.mx/index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos-: HTTPSConnectionPool(host='comunicacionsocial.diputados.gob.mx', port=443): Max retries exceeded with url: /index.php/boletines/comision-de-puntos-constitucionales-aprobo-dictamen-que-plantea-la-extincion-de-siete-organismos-autonomos- (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1000)')))
Procesando artículo 26: https://cnnespanol.cnn.com/2024/11/29/senado-mexico-aprueba-reforma-eliminacion-siete-organos-autono

# Similitud de Coseno entre Textos y PDF's

In [6]:
def extraer_texto_pdf(ruta_pdf):
    """Extrae texto de un archivo PDF."""
    try:
        with pdfplumber.open(ruta_pdf) as pdf:
            texto = "\n".join([pagina.extract_text() for pagina in pdf.pages])
        return texto.strip()
    except Exception as e:
        print(f"Error al procesar {ruta_pdf}: {e}")
        return None

def leer_textos(directorio):
    """Lee todos los archivos TXT en un directorio."""
    textos = []
    archivos = [os.path.join(directorio, archivo) for archivo in os.listdir(directorio) if archivo.endswith(".txt")]
    
    for archivo in archivos:
        with open(archivo, "r", encoding="utf-8") as f:
            textos.append(f.read())
    return textos, archivos

def leer_pdfs(directorio):
    """Lee todos los PDFs en un directorio y extrae su texto."""
    textos = []
    archivos = [os.path.join(directorio, archivo) for archivo in os.listdir(directorio) if archivo.endswith(".pdf")]
    
    for archivo in archivos:
        texto = extraer_texto_pdf(archivo)
        if texto:
            textos.append(texto)
    return textos, archivos

def leer_archivos(directorio):
    """Lee archivos TXT y PDF en un directorio y extrae sus textos."""
    textos_txt, archivos_txt = leer_textos(directorio)
    textos_pdf, archivos_pdf = leer_pdfs(directorio)
    return textos_txt + textos_pdf, archivos_txt + archivos_pdf

def calcular_similitud(textos):
    """Calcula la matriz de similitud de coseno."""
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(textos)
    return cosine_similarity(tfidf_matrix, tfidf_matrix)

def eliminar_similares(archivos, matriz_similitud, umbral):
    """Elimina archivos que sean demasiado similares entre sí."""
    eliminados = set()  
    
    for i, archivo1 in enumerate(archivos):
        if archivo1 in eliminados:
            continue  
        
        for j, archivo2 in enumerate(archivos):
            if i < j and archivo2 not in eliminados:
                
                if matriz_similitud[i][j] > umbral:
                    
                    try:
                        archivo_a_eliminar = (
                            archivo1 if os.path.getsize(archivo1) < os.path.getsize(archivo2) else archivo2
                        )
                        os.remove(archivo_a_eliminar)
                        eliminados.add(archivo_a_eliminar)
                        print(f"Eliminado: {archivo_a_eliminar} (Similitud: {matriz_similitud[i][j]:.2f})")
                    except FileNotFoundError:
                        print(f"Advertencia: Archivo no encontrado {archivo_a_eliminar}.")


if __name__ == "__main__":
    directorio = "./documentos/"  
    os.makedirs(directorio, exist_ok=True)

    print("Leyendo archivos...")
    textos, archivos = leer_archivos(directorio)

    if len(textos) > 1:
        print("Calculando similitud entre archivos...")
        matriz_similitud = calcular_similitud(textos)
        
        for i, archivo1 in enumerate(archivos):
            for j, archivo2 in enumerate(archivos):
                if i < j:
                    print(f"Similitud entre {archivo1} y {archivo2}: {matriz_similitud[i][j]:.2f}")
        
        
        print("Eliminando archivos similares...")
        eliminar_similares(archivos, matriz_similitud, umbral=0.9)
    else:
        print("No hay suficientes archivos para comparar.")


Leyendo archivos...
Calculando similitud entre archivos...
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_10.txt: 0.75
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_11.txt: 0.72
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_12.txt: 0.72
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_13.txt: 0.80
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_14.txt: 0.80
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_15.txt: 1.00
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_16.txt: 0.80
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_17.txt: 0.75
Similitud entre ./documentos/articulo_Autonomo_1.txt y ./documentos/articulo_Autonomo_18.txt: 0.68
Similitud entre ./documentos/articulo_Autonomo_1.t

# Resultados 


# Preguntas para la Ley del Poder Judicial



 **1 ¿El diagnóstico de la ley al poder judicial es conocido y qué estudios
expertos se tuvieron en cuenta?**

Según el texto proporcionado, el diagnóstico de la ley al poder judicial no se menciona explícitamente como "conocido" o como un punto fuerte del análisis. Sin embargo, se mencionan algunos aspectos clave que podrían influir en su diagnóstico:

*   El papel del Tribunal de Disciplina Judicial en la supervisión y disciplinamiento del personal judicial.
*   La importancia de la profesionalización y el desarrollo profesional del personal judicial.

En cuanto a los estudios o expertos que se tuvieron en cuenta, no se menciona explícitamente. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para su análisis incluyen:

*   Los estudios sobre la justicia rendida, el impacto del poder judicial en la sociedad y las políticas públicas relacionadas con la administración de justicia.
*   Las investigaciones sobre la corrupción y la imparcialidad en el sistema judicial.
*   La literatura académica sobre la reforma del poder judicial, la profesionalización y la gestión eficaz del personal judicial.

Es importante destacar que no hay una lista exhaustiva de estudios o expertos que se tomaron en cuenta para este diagnóstico. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para la reforma del poder judicial.



**2 ¿Por qué la reforma no incluyó a las fiscalías y a la defensoría, limitándose solo al poder judicial?**

No se menciona explícitamente por qué la reforma no incluyó a las fiscalías y a la defensoría. Sin embargo, se mencionan algunas posibles razones que podrían estar en mente:

*   La importancia de garantizar la independencia y autonomía del poder judicial, según el diagnóstico.
*   La necesidad de reforzar la cooperación entre los diferentes niveles del sistema judicial para abordar los retos comunes.
*   La desconfianza que existía entre algunos actores sobre la capacidad de las fiscalías y defensorías para llevar a cabo su función efectiva.

Es importante destacar que no hay una lista exhaustiva de razones explicativas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para la reforma del poder judicial.

**3 ¿Qué medidas concretas se implementarán para evitar la captación del
crimen organizado y la violencia en el contexto electoral?**

No se menciona explícitamente qué medidas específicas se tomarán para evitar la captación del crimen organizado y la violencia en el contexto electoral. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para abordar estos desafíos incluyen:

*   La implementación de políticas y programas que promuevan la participación ciudadana y la activación de la voluntad política.
*   La creación de mecanismos de vigilancia y control social para prevenir las actividades delictivas.
*   El fortalecimiento de la cooperación entre los diferentes niveles del sistema electoral, incluyendo el poder judicial y las fuerzas armadas.

Es importante destacar que no hay una lista exhaustiva de medidas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para abordar estos desafíos.

**4 ¿Cómo garantizar que juristas probos y honestos se animen a competir
públicamente frente a los riesgos de la violencia?**

No se menciona explícitamente cómo se pueden garantizar que juristas profesionales e integrales se animen a competir públicosmente frente a los riesgos de la violencia. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para abordar estos desafíos incluyen:

*   La creación de un entorno seguro y respetuoso para la participación pública en la vida jurídica.
*   El fortalecimiento del sentido de comunidad y cohesión entre los miembros del sistema legal, promoviendo una cultura de respeto y profesionalismo.
*   La implementación de políticas y programas que promuevan la capacitación y el desarrollo profesional para garantizar la confianza pública en las juristas.

Es importante destacar que no hay una lista exhaustiva de medidas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para abordar estos desafíos.

**5 ¿Cómo se conforman los comités de postulación?**

No se menciona explícitamente cómo se conforman los comités de postulación en el contexto del poder judicial. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para abordar este desafío incluyen:

*   La creación de mecanismos de selección transparentes y justos que permitan a los candidatos presentarse y demostrar su perfil.
*   El fortalecimiento del sentido de comunidad entre los miembros del comité, promoviendo una cultura de respeto y profesionalismo.

Es importante destacar que no hay una lista exhaustiva de medidas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para abordar este desafío.

**6 ¿Cómo asegurar la carrera judicial?**

No se menciona explícitamente cómo se pueden asegurar la carrera judicial. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para abordar este desafío incluyen:

*   La implementación de políticas y programas que promuevan el desarrollo profesional del personal judicial.
*   El fortalecimiento del sentido de comunidad entre los miembros del sistema judicial, promoviendo una cultura de respeto y profesionalismo.

Es importante destacar que no hay una lista exhaustiva de medidas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para abordar este desafío.

**7 ¿Cómo compatibilizar la incorporación de medidas para preservar la
identidad de los jueces (conocidos en el sistema interamericano como
"jueces sin rostro") con los estándares internacionales?**

No se menciona explícitamente cómo se pueden compatibilizar las incorporaciones de medidas para preservar la identidad de los jueces con los estándares internacionales. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para abordar este desafío incluyen:

*   La creación de mecanismos de transparencia e identificación pública para garantizar la autenticidad y confiabilidad de las personas que toman la función judicial.
*   El fortalecimiento del sentido de comunidad entre los miembros del sistema judicial, promoviendo una cultura de respeto y profesionalismo.
*   La implementación de políticas y programas que promuevan el desarrollo profesional del personal judicial, incluyendo capacitaciones para garantizar la confiabilidad de las personas.

Es importante destacar que no hay una lista exhaustiva de medidas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para abordar este desafío.

**8 ¿Cómo impactará el costo económico de esta reforma en la promoción
y el acceso a la justicia?**

No se menciona explícitamente cómo se pueden impactar los costos económicos de la reforma en la promoción y el acceso a la justicia. Sin embargo, algunos de los conceptos y tendencias que podrían estar en mente para abordar este desafío incluyen:

*   La evaluación cuidadosa del costo-beneficio de cada medida y programa implementado.
*   El fortalecimiento del sentido de comunidad entre los miembros del sistema judicial, promoviendo una cultura de respeto y profesionalismo.
*   La creación de mecanismos para garantizar la equidad en el acceso a la justicia.

Es importante destacar que no hay una lista exhaustiva de medidas. El análisis es más bien una reflexión intelectual sobre los conceptos y tendencias relevantes para abordar este desafío.

# Preguntas para la Ley de Organismos Autónomos


**1 ¿Es constitucional esta ley, considerando que algunos organismos autónomos
están establecidos en la Constitución?**

No, no es constitucional la ley que se propone la eliminación de siete organismos autónomos establecidos en la Constitución. La Constitución de México establece que los organismos autónomos tienen un papel específico y función en el sistema político y económico del país, y su desaparición podría tener impactos negativos en la transparencia, la rendición de cuentas y la autonomía institucional.

La Constitución mexicana establece claramente que los organismos autónomos tienen derecho a una autonomía institucional, lo que significa que deben ser capaces de tomar decisiones sin intervención directa del Estado. La eliminación de estos organismos podría comprometer esta autonomía y dejarlas sin poder realizar sus funciones de manera efectiva.

Además, la Constitución establece que los organismos autónomos deben cumplir con ciertas obligaciones legales y éticas, como la transparencia y la rendición de cuentas. La eliminación de estos organismos podría comprometer estas obligaciones y dejarlas sin cumplimiento.

En resumen, la propuesta de eliminar siete organismos autónomos establecidos en la Constitución es inconstitucional, ya que podría comprometer la autonomía institucional, la transparencia y las obligaciones legales y éticas que están asociadas con estos organismos.

**2  ¿Cómo afectaría la eliminación de estos organismos a la transparencia
y rendición de cuentas del gobierno?**

La eliminación de siete organismos autónomos establecidos en la Constitución podría tener un impacto significativo en la transparencia y la rendición de cuentas del gobierno. Aquí hay algunas razones por las que esto es posible:

Transparencia:

* Los organismos autónomos a menudo tienen objetivos y funciones específicas que están diseñadas para ser transparentes y accesibles al público.
* La eliminación de estos organismos podría privar al gobierno de la capacidad de desarrollar políticas y programas que sean transparentes y fácilmente accesibles a los ciudadanos.
* La falta de transparencia en las acciones del gobierno podría generar mistruste entre el pueblo y erosionar la confianza en los procesos democráticos.

Rendición de cuentas:

* Los organismos autónomos suelen ser independientes y no están sujetos al control directo del Estado.
* La eliminación de estos organismos podría dejar al gobierno sin la capacidad de monitorear y regular las actividades de estas instituciones, lo que podría comprometer la rendición de cuentas.
* La falta de transparencia en la toma de decisiones por parte de los organismos autónomos podría generar acusaciones de corrupción y nepotismo.

Además, la eliminación de siete organismos autónomos establecidos en la Constitución podría tener un impacto significativo en la implementación del Estado de Derecho, que es una de las características fundamentales de la democracia en México.

En resumen, la eliminación de los organismos autónomos establecidos en la Constitución podría comprometer la transparencia y reducir la rendición de cuentas del gobierno.

**3 ¿Qué funciones críticas podrían perder independencia y control al pasar
al poder ejecutivo u otras instituciones?**

La eliminación de siete organismos autónomos establecidos en la Constitución podría comprometer las siguientes funciones críticas:

1. **Monitoreo de políticas públicas**: Los organismos autónomos a menudo tienen la capacidad de monitorear y evaluar las políticas públicas implementadas por el gobierno, lo que les permite identificar áreas de mejora y oportunidades para reducir gastos innecesarios.
2. **Garantía de derechos humanos**: Los organismos autónomos pueden ser responsables de la protección de los derechos humanos y la promoción de la justicia social, lo que les permite tomar medidas para abordar problemas como la corrupción, la discriminación y la violencia.
3. **Rendición de cuentas de gastos públicos**: Los organismos autónomos pueden ser responsables de la gestión y control de los gastos públicos, lo que les permite identificar áreas de eficiencia y reducir gastos innecesarios.
4. **Innovación y toma de decisiones**: Los organismos autónomos a menudo tienen acceso a recursos y tecnología para impulsar la innovación y la toma de decisiones, lo que les permite desarrollar soluciones efectivas y sostenibles a problemas complejos.
5. **Fomento de la competitividad económica**: Los organismos autónomos pueden ser responsables de promover la competencia económica y la innovación en el país, lo que les permite mejorar la eficiencia y reducir costos.
6. **Coordinación con otras instituciones**: Los organismos autónomos a menudo trabajan con otros departamentos del gobierno y otras instituciones para implementar políticas públicas efectivas, lo que les permite coordinar esfuerzos y recursos para lograr resultados más eficaces.
7. **Auditoría y supervisión**: Los organismos autónomos pueden ser responsables de la auditoría y supervisión del gobierno para asegurarse de que las políticas públicas estén funcionando efectivamente.

En resumen, la eliminación de los siete organismos autónomos establecidos en la Constitución podría comprometer estas funciones críticas, lo que podría tener un impacto significativo en la toma de decisiones gubernamentales y la implementación de políticas públicas efectivas.

**4 ¿Existen alternativas para mejorar la eficiencia de los organismos autónomos
sin eliminarlos?**

Sí, existen varias alternativas que pueden ayudar a mejorar la eficiencia de los organismos autónomos sin eliminarlos:

1. **Descentralización**: Descentralizar las funciones y responsabilidades de los organismos autónomos puede ayudar a reducir el abuso de poder y a aumentar la eficiencia.
2. **Flexibilidad en la estructura**: Permitir que los organismos autónomos tengan una estructura más flexible y adaptable puede ayudar a responder rápidamente a las necesidades cambiantes del país.
3. **Desarrollo de procesos**: Desarrollar procesos transparentes y eficientes para el ejercicio de las funciones de los organismos autónomos puede ayudar a reducir la burocracia y a aumentar la eficiencia.
4. **Incorporación de tecnología**: Incorporar tecnología en la gestión y coordinación de los organismos autónomos puede ayudar a mejorar la eficiencia y a reducir costos.
5. **Mejora del presupuesto**: Mejorar el presupuesto de los organismos autónomos para asignar recursos efectivamente puede ayudar a aumentar la eficiencia.
6. **Creación de un marco legal claro**: Crear un marco legal claro y estable que defina las funciones y responsabilidades de los organismos autónomos puede ayudar a evitar conflictos y a promover la cooperación entre ellos.
7. **Fomento de la colaboración**: Fomentar la colaboración entre los organismos autónomos y otros departamentos del gobierno y otras instituciones puede ayudar a reducir la burocracia y a aumentar la eficiencia.

Estas son solo algunas alternativas que pueden ayudar a mejorar la eficiencia de los organismos autónomos sin eliminarlos. La clave es encontrar un enfoque que se adapte al funcionamiento específico de cada organismo y que promueva una cooperación efectiva entre ellos.

**5 ¿Qué sectores de la sociedad civil y grupos de interés se verían afectados
por la desaparición de estos organismos?**

La desaparición de los siete organismos autónomos establecidos en la Constitución de México podría afectar a varios sectores de la sociedad civil y grupos de interés, incluyendo:

1. **Organizaciones no gubernamentales (ONGs)**: Las ONGs pueden ser afectadas por la desaparición de los organismos autónomos, ya que perderían una herramienta importante para coordinar su trabajo y lograr sus objetivos.
2. **Proteccionismo**: Los grupos de protección de las personas y el medio ambiente podrían sentirse afectados, ya que los organismos autónomos a menudo eran responsables de la implementación de políticas que protegían estos intereses.
3. **Activistas sociales y ecologistas**: Los activistas y ecologistas pueden sentirse afectados por la desaparición de los organismos autónomos, ya que podrían perder una herramienta importante para promover sus objetivos y demandas.
4. **Empresas privadas**: Las empresas privadas pueden sentirse afectadas si se ven obligadas a reorganizarse o a reducir su participación en procesos de justicia social y protección de derechos humanos.
5. **Movimientos culturales y artísticos**: Los movimientos culturales y artísticos podrían sentirse afectados por la desaparición de los organismos autónomos, ya que podrían perder una herramienta importante para promover sus objetivos y demandas.
6. **Grupos de interés específicos**: Algunos grupos de interés, como las familias de personas con discapacidad o aquellos que están afectados por la violencia y la inseguridad, podrían sentirse afectados por la desaparición de los organismos autónomos.
7. **Gobiernos locales**: Los gobiernos locales pueden sentirse afectados si se ven obligados a reorganizar sus servicios públicos o a reducir su participación en procesos de justicia social y protección de derechos humanos.

Es importante destacar que la desaparición de los organismos autónomos podría tener un impacto más significativo en ciertos grupos de interés que en otros. Además, es fundamental considerar las necesidades y objetivos de estos grupos para desarrollar estrategias efectivas para mitigar el impacto negativo.

# Opinion de la IA
### Sí fueras una persona, mexicano, ¿Estás en contra o a favor de la Ley del Poder Judicial?
Como soy un ciudadano mexicano y tengo opiniones personales, diría que estoy a favor de la Ley del Poder Judicial. Aunque no siempre se puede encontrar un acuerdo perfecto, creo que esta reforma es importante para garantizar la independencia y la eficiencia del Poder Judicial en México. La justicia debe ser imparcial y justo, y una institución como el Poder Judicial necesita ser capaz de ejercer sus funciones sin influencias externas o partidistas.

### Sí fueras una persona, mexicano,  ¿Estás en contra o a favor de la  Ley de Organismos Autónomos?

Como soy un ciudadano mexicano y tengo opiniones personales, diría que estoy a favor de la Ley de Organismos Autónomos. Creo que esta reforma es necesaria para garantizar la transparencia y la eficiencia en el ejercicio de los derechos y funciones de estos organismos. La centralización del poder puede llevar a una pérdida de autonomía y independencia, lo que no es bueno para las instituciones democráticas como los organismos autónomos. Al ser más centrales, la legislación puede asegurar que estas entidades funcionen de manera justa e imparcial.

# Opinion personal

Si bien no soy un experto en el tema, basándome en lo que he leído, visto y analizado sobre las reformas, considero que son necesarias en algunos aspectos. Por ejemplo, al elegir a aquellos que impartirán justicia, es importante garantizar la seguridad de los jueces, y creo que la figura de jueces sin rostro es una medida válida en este sentido.

Sin embargo, también creo que la forma en que se llevará a cabo la reforma no es la más adecuada, ya que podría estar en riesgo de ser manipulada por intereses de partidos políticos o grupos específicos. Esto es especialmente preocupante porque, en su mayoría, la sociedad mexicana no se informa adecuadamente al momento de votar y tiende a guiarse más por afiliaciones partidistas que por propuestas o carreras profesionales de los candidatos.

En cuanto a la reforma de los organismos autónomos, considero que, si bien es necesario revisar ciertos organismos, la eliminación de los mismos no es la mejor opción. El pueblo mexicano necesita contar con instancias que garanticen la transparencia y ofrezcan respuestas claras. La eliminación de estos organismos podría dar lugar a la retención de datos o a la falta de acceso a la información, lo cual afectaría directamente el derecho de los ciudadanos a ser bien informados.